In [1]:
#Importation des librairies nécessaires au déroulement du projet
import requests
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import mstats
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression

In [ ]:
#Gestion des doublons
def gerer_doublons(data, keep="first"):
    # Détection des doublons
    doublons = data.duplicated(subset=colonnes_doublons, keep=keep)
    
    # Suppression des doublons
    data = data[~doublons]
    return data


### Gestion des valeurs manquantes (cas analyse univarié)
- Si pour une variable donnée, on a peu de valeurs manquantes, on travail avec le jeu de données comme tel
- Si par contre, on a beaucoup de valeurs manquantes dans une varible donnée, soit on l'oubliera si elle n'est pas importante pour notre analyse, soit on créera un sous-échantillon et on supprimera les individus pour lesquelles cette variable est manquante si elle est cruciale pour notre analyse.
- On peut également passer par l'imputation qui est le fait de deviner une valeur manquante
L'imputation par la moyenne, la médiane et le mode sont des méthodes simples pour remplacer les valeurs manquantes dans une variable.

L'imputation par la moyenne consiste à remplacer les valeurs manquantes par la moyenne des valeurs non manquantes dans la variable. Cette méthode est utile pour les variables continues et symétriques.

L'imputation par la médiane consiste à remplacer les valeurs manquantes par la médiane des valeurs non manquantes dans la variable. Cette méthode est utile pour les variables continues qui ont une distribution asymétrique ou pour les variables ordinales.

L'imputation par le mode consiste à remplacer les valeurs manquantes par le mode, c'est-à-dire la valeur la plus fréquente, dans la variable. Cette méthode est utile pour les variables catégorielles ou nominales.



In [11]:
def manage_missing_values(data):
    #Seuil pour la suppression des lignes
    seuil_suppression = len(data.columns) // 2
    data2 = data
    
    # Parcourir chaque colonne pour effectuer l'imputation et la suppression
    for col in data2.columns:
        # Vérifier si le nombre de valeurs manquantes dans la colonne dépasse le seuil de suppression
        if data[col].isnull().sum() > seuil_suppression:
            # Suppression des lignes avec des valeurs manquantes dans cette colonne
            data = data.dropna(subset=[col])
        elif data[col].isnull().sum() < seuil_suppression and data[col].isnull().sum() != 0:
            # Imputation par le mode si les valeurs manquantes sont minimes
            mode_value = data[col].mode()[0]
            data[col].fillna(mode_value, inplace=True)
        else:
            continue

    return data

   properties.gml_id properties.partition_field  properties.code_no2  \
0          875183780                   282024w4                    1   
1          971927298                   282024w8                    1   
2          971929932                   282024w9                    1   
3          971932679                   282024w9                    2   

   properties.code_o3  properties.code_pm10  properties.code_pm25  \
0                   2                     1                     1   
1                   2                     1                     1   
2                   2                     1                     1   
3                   2                     1                     1   

   properties.code_qual  properties.code_so2 properties.code_zone  \
0                     2                    1                14118   
1                     2                    1                14118   
2                     2                    1                14118   
3                

### Gestion des valeurs manquantes (cas analyse bivarié)
Pour traiter les valeurs manquantes pour une variable quantitative dépendante d'une variable qualitative, on peut utiliser la méthode d'imputation par groupe. On remplace les valeurs manquantes par des statistiques calculées à partir des valeurs non manquantes du même groupe de la variable qualitative.

Pour traiter les valeurs manquantes pour une variable quantitative dépendante d'une autre variable quantitative, on peut utiliser plusieurs approches. En voici deux exemples :
- Imputation par la moyenne ou la médiane :
Cette méthode consiste à remplacer les valeurs manquantes par la moyenne ou la médiane des valeurs non manquantes de la même variable quantitative.
- Imputation basée sur la régression :
Dans cette approche, on peut utiliser une régression linéaire pour prédire les valeurs manquantes de la variable quantitative dépendante en fonction des valeurs non manquantes de l'autre variable quantitative.

Pour traiter les valeurs manquantes pour une variable qualitative dépendante d'une autre variable qualitative, on peut utiliser une méthode de régression logistique. La régression logistique est couramment utilisée pour la classification binaire, mais elle peut également être utilisée pour traiter les valeurs manquantes dans les variables qualitatives.

In [ ]:
def impute_missing_values(data, variable_qualitative, variable_quantitative):
    # Calculer les statistiques par groupe
    statistics_by_group = data.groupby(variable_qualitative)[variable_quantitative].agg(['mean', 'median', 'mode']).reset_index()
    
    # Fonction d'imputation par groupe
    def impute_group(row):
        if pd.isnull(row[variable_quantitative]):
            mean_value = statistics_by_group.loc[statistics_by_group[variable_qualitative] == row[variable_qualitative], 'mean'].values[0]
            median_value = statistics_by_group.loc[statistics_by_group[variable_qualitative] == row[variable_qualitative], 'median'].values[0]
            mode_value = statistics_by_group.loc[statistics_by_group[variable_qualitative] == row[variable_qualitative], 'mode'].values[0]
            return mode_value if pd.notnull(mode_value) else (median_value if pd.notnull(median_value) else mean_value)
        else:
            return row[variable_quantitative]
    
    # Appliquer la fonction d'imputation par groupe
    data[variable_quantitative] = data.apply(impute_group, axis=1)
    
    return data

In [ ]:
def impute_missing_values_regression(data, dependent_variable, independent_variable):
    # Séparation des données en deux ensembles : avec valeurs manquantes et sans valeurs manquantes
    data_with_missing = data[data[dependent_variable].isnull()]
    data_without_missing = data[data[dependent_variable].notnull()]
    
    # Ajustement du modèle de régression linéaire
    model = LinearRegression()
    model.fit(data_without_missing[[independent_variable]], data_without_missing[dependent_variable])
    
    # Prédiction des valeurs manquantes
    predicted_values = model.predict(data_with_missing[[independent_variable]])
    data_with_missing[dependent_variable] = predicted_values
    
    # Concaténation des données
    data_imputed = pd.concat([data_with_missing, data_without_missing])
    
    return data_imputed

In [ ]:
def impute_missing_values_logistic(data, dependent_variable, independent_variable):
    # Séparation des données en deux ensembles : avec valeurs manquantes et sans valeurs manquantes
    data_with_missing = data[data[dependent_variable].isnull()]
    data_without_missing = data[data[dependent_variable].notnull()]
    
    # Encodage des variables catégorielles pour la régression logistique
    X = pd.get_dummies(data_without_missing[independent_variable])
    y = data_without_missing[dependent_variable]
    
    # Ajustement du modèle de régression logistique
    model = LogisticRegression(solver='lbfgs', multi_class='auto')
    model.fit(X, y)
    
    # Prédiction des valeurs manquantes
    predicted_values = model.predict(pd.get_dummies(data_with_missing[independent_variable]))
    data_with_missing[dependent_variable] = predicted_values
    
    # Concaténation des données
    data_imputed = pd.concat([data_with_missing, data_without_missing])
    
    return data_imputed

### Gestion des valeurs aberrantes
On dispose de plusieurs méthodes pour identifier les valeurs aberrantes:
- L'IQR: La méthode de l'IQR est une approche courante pour identifier les valeurs aberrantes dans des données univariées. Elle utilise l'écart interquartile pour définir une plage de valeurs plausibles et identifie les observations qui sont en dehors de cette plage.
- Le Z-score: La méthode du z-score est une autre approche courante pour identifier les valeurs aberrantes dans des données univariées. Elle utilise l'écart à la moyenne pour définir une plage de valeurs plausibles et identifie les observations qui sont en dehors de cette plage.

Pour gérer ces valeurs, nous devons prendre en compte deux cas:

|Type de valeur aberrante|Solutions pour les traiter|
|------------------------|--------------------------|
|Erreur|1. Correction de l'erreur si possible (ex. erreurs de saisie)|
||2. Suppression de l'observation si la correction n'est pas possible et si l'erreur est clairement une anomalie|
||3. Remplacer par une valeur manquante et imputer avec les techniques d'imputation|
|Extrême (Plus notre cas)|1. Transformation des données (ex. logarithmique, racine carrée) pour réduire l'impact des valeurs extrêmes|
||2. Winsorisation : remplacer les valeurs extrêmes par des percentiles spécifiques (ex. 1er et 99e percentiles)|
||3. Utiliser des modèles d'apprentissage automatique robustes aux valeurs extrêmes (ex. modèles basés sur les rangs)|
||4. Discrétiser la variable|
||5. Faire le modèle avec et sans les valeurs extrêmes et analyser l'impact|

In [ ]:
# Fonction pour détecter et gérer les outliers en utilisant l'IQR et la winsorisation
def detect_outliers_iqr(data):
    for col in data.columns:
        Q1 = data[col].quantile(0.25)
        Q3 = data[col].quantile(0.75)
        IQR = Q3 - Q1
        
        # Valeurs de winsorisation
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        
        # Remplacement des outliers
        data[col] = data[col].where(data[col] > lower_bound, lower_bound)
        data[col] = data[col].where(data[col] < upper_bound, upper_bound)
    return data
